In [1]:
# required packages:
# langchain
# openai
# tiktoken
# chromadb

import os
import logging
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chat_models import AzureChatOpenAI
from langchain.indexes import VectorstoreIndexCreator
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.vectorstores import Chroma
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
    Language,
)

In [2]:
os.environ['OPENAI_API_TYPE'] = 'azure'
os.environ['OPENAI_API_VERSION'] = '2023-03-15-preview'
# os.environ['OPENAI_API_BASE'] = 'https://###.openai.azure.com/'
# os.environ['OPENAI_API_KEY'] = '###'

In [3]:
# azure openai

azure_openai_embeddings = OpenAIEmbeddings(model='text-embedding-ada-002', chunk_size=1, disallowed_special=())

azure_chat_openai = AzureChatOpenAI(deployment_name='gpt-4')

In [ ]:
# load code

# https://github.com/Azure/azure-rest-api-specs-examples
# only use apimanagement, about 400 java files
repo_dir = 'C:\\github\\azure-rest-api-specs-examples\\specification\\apimanagement\\resource-manager\\Microsoft.ApiManagement\\stable\\2021-08-01\\examples-java'

loaders = []
for dirpath, dirnames, filenames in os.walk(repo_dir):
    for filename in filenames:
        if filename.endswith('.java'):
            loader = TextLoader(os.path.join(dirpath, filename), encoding='utf8')
            loaders.append(loader)
            
# index repository

index_creator = VectorstoreIndexCreator(
    vectorstore_cls=Chroma, 
    embedding=azure_openai_embeddings,
    text_splitter=RecursiveCharacterTextSplitter.from_language(language=Language.JAVA),
    vectorstore_kwargs={
        'persist_directory': './db-api-management'
    }
)

index = index_creator.from_loaders(loaders)

# persist vectordb to disk
index.vectorstore.persist()

In [4]:
# or load existing vectordb from disk

vectordb = Chroma(persist_directory='./db-api-management', embedding_function=azure_openai_embeddings)
index = VectorStoreIndexWrapper(vectorstore=vectordb)

In [9]:
vectordb = index.vectorstore
print('vector store collection size: ' + str(vectordb._collection.count()))

retriever = vectordb.as_retriever()

qa = RetrievalQA.from_chain_type(llm=azure_chat_openai, chain_type='stuff', retriever=retriever)

vector store collection size: 388


In [9]:
answer = qa.run('how to create api management')
print(answer)

To create an API Management instance using the Azure SDK for Java, you can follow the sample codes provided in the context. Here's a general outline of the steps to create an API Management instance:

1. Import the necessary packages.
2. Create a class with methods to create different types of APIs.
3. In each method, use the `ApiManagementManager` object to define and create the API.

Here's an example to create a simple API:

```java
import com.azure.resourcemanager.apimanagement.models.AuthenticationSettingsContract;
import com.azure.resourcemanager.apimanagement.models.Protocol;
import com.azure.resourcemanager.apimanagement.models.SubscriptionKeyParameterNamesContract;
import java.util.Arrays;

public final class Main {
    public static void apiManagementCreateApi(com.azure.resourcemanager.apimanagement.ApiManagementManager manager) {
        manager
            .apis()
            .define("tempgroup")
            .withExistingService("rg1", "apimService1")
            .withDispl

In [10]:
answer = qa.run('how to configure gateway')
print(answer)

To configure a gateway in Azure API Management, you can follow these steps using the provided sample code:

1. First, import the necessary classes at the beginning of your file:

```java
import com.azure.core.util.Context;
import com.azure.resourcemanager.apimanagement.ApiManagementManager;
import com.azure.resourcemanager.apimanagement.models.GatewayContract;
import com.azure.resourcemanager.apimanagement.models.ResourceLocationDataContract;
```

2. Create or update an existing gateway with the desired configuration:

```java
public static void configureGateway(ApiManagementManager manager, String resourceGroupName, String serviceName, String gatewayName) {
    // Retrieve the existing gateway
    GatewayContract resource = manager.gateways().getWithResponse(resourceGroupName, serviceName, gatewayName, Context.NONE).getValue();

    // Update the gateway with new configuration
    resource
        .update()
        .withLocationData(new ResourceLocationDataContract().withName("my loca

In [11]:
answer = qa.run('how to create logger')
print(answer)

# notes: it actually find the 2 samples of logger configure

To create a logger, you can follow the sample code provided below:

For Application Insights Logger:
```java
import com.azure.resourcemanager.apimanagement.ApiManagementManager;
import com.azure.resourcemanager.apimanagement.models.LoggerType;
import java.util.HashMap;
import java.util.Map;

public class Main {
    public static void main(String[] args) {
        ApiManagementManager manager = ...; // Initialize the ApiManagementManager instance

        apiManagementCreateAILogger(manager);
    }

    public static void apiManagementCreateAILogger(ApiManagementManager manager) {
        manager
            .loggers()
            .define("loggerId")
            .withExistingService("rg1", "apimService1")
            .withLoggerType(LoggerType.APPLICATION_INSIGHTS)
            .withDescription("adding a new logger")
            .withCredentials(mapOf("instrumentationKey", "11................a1"))
            .create();
    }

    private static <T> Map<String, T> mapOf(Object... inputs)

In [12]:
answer = qa.run('what resource do I need to create, before I can run api release')
print(answer)

# notes: it actually replies what are the depencencies, which is not clear from the samples.
# It would be great if follow up question like "what are the code to create them" can be answered correctly.

Before you can run API release, you need to create an API Management service instance and an API within that instance. In the code samples provided, the API Management service instance is referred to as "apimService1" and the API is referred to as "a1". These resources should be created within a resource group, in this case, "rg1".


In [14]:
answer = qa.run('how to create api management, and all required dependencies, till code to run api release')
print(answer)

# notes: not sure if all code compiles, but it looks about right.

To create an API Management instance and all required dependencies, you need to follow these steps:

1. Set up your development environment and authenticate with Azure. You can refer to the Azure SDK for Java documentation on how to do this: https://docs.microsoft.com/en-us/azure/developer/java/sdk/java-sdk-configure-environment

2. Create an API Management instance in Azure. You can use the following code snippet as a reference:

```java
import com.azure.resourcemanager.AzureResourceManager;
import com.azure.resourcemanager.apimanagement.models.ApiManagementService;
import com.azure.core.management.Region;

public static void createApiManagementInstance(AzureResourceManager azureResourceManager) {
    String resourceGroupName = "rg1";
    String apimServiceName = "apimService1";
    String location = Region.US_WEST.name();

    ApiManagementService apiManagementService = azureResourceManager.apiManagementServices().define(apimServiceName)
            .withRegion(location)
            